In [ ]:
from conda.plugins.subcommands.doctor import execute
#Import necessary Libraries
from flask import Flask, render_template, request, redirect, url_for
import sqlalchemy
import flask_sqlalchemy
import sqlite3
import os
from threading import Thread
from IPython.display import display, Javascript
from werkzeug.security import generate_password_hash, check_password_hash


#Initialise the Flask application
app = Flask(__name__)

#Store Registered users in memory (for simplicity)
users = []


#Function to connect to the SQLite database and initialise the table
def init_db():
    conn = sqlite3.connect('users1.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
    conn.commit()   
    conn.close()

#Route for the Home Page
@app.route('/')
def home():
    return render_template('home.html')

#Route for Logged In Page
@app.route('/logged_in')
def logged_in():
    return render_template('logged_in.html')

#Route for the About page
@app.route('/about')
def about():
    return render_template('about.html')

#Route for the Contact Us Page
@app.route('/contact')
def contact():
    return render_template('contact.html')

#Route to display registered users
@app.route('/users')
def users():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT username, created_at FROM users")
    users = cursor.fetchall()
    conn.close()
    
    return render_template('users.html', users=users)

#Route for the Registration Page
#@app.route('/register', methods=['GET', 'POST'])
#def register():
   # if request.method == 'POST':
      # username = request.form['username']
     #password = request.form['password']
        
        #Insert the new user into the database
        #conn = sqlite3.connect('users.db')
       # cursor = conn.cursor()
       # cursor.execute("Insert INTO users (username, password) VALUES (?,?)", (username,password))
       # conn.commit()
       # conn.close()
    
       # return redirect('/users')
   # return render_template('register.html')

#Route to handle user registration
@app.route('/register', methods=['POST'])
def do_register():
    first_name = request.form['first_name']
    last_name = request.form['last_name']
    email = request.form['email']
    password = request.form['password']
    
    #Hash the password before saving to the database
    #hashed password method is sha256
    hashed_password = generate_password_hash(password, method=' pbkdf2:sha256')
    
    #Insert the new user into the database
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor = execute('''
        INSERT INTO users (first_name, last_name, email, password)
        VALUES (?, ?, ?, ?)
        '''), (first_name, last_name, email, hashed_password)
    conn.commit()
    conn.close()
    
    return redirect('/users')
        
#Route for the Login page
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
    
        #try to match the login against the passed in values
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT() FROM users WHERE username=? AND password=?", (username, password))
        matches = cursor.fetchone()[0]
        conn.close()
    
        if matches == 1:
            return redirect(url_for('logged_in'))
    
    
    return render_template('login.html')
        
def run_flask():
    #Display a link to open the Flask app in a new browser tab
    #display(Javascript('window.open("/proxy/5000/","_blank")'))
    app.run(host='0.0.0.0', port=5000)
    
# Start Flask app in a thread
Thread(target=run_flask).start()

#start the flask application
init_db()
run_flask()


 * Serving Flask app '__main__'
 * Debug mode: off
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.118:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Nov/2024 11:21:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2024 11:21:30] "GET /static/images/hospital.png HTTP/1.1" 304 -
127.0.0.1 - - [05/Nov/2024 11:21:32] "GET /register HTTP/1.1" 405 -
